# Extração de dados do TMDB (The Movie Database)
---
27/09/2025

Este notebook tem como objetivo construir um dataset de filmes utilizando a API do The Movie Database (TMDB). O processo é realizado em duas etapas interligadas:
1. A primeira etapa é a extração dos dados de **índice** dos filmes a partir do endpoint `/discover/movie`
2. A segunda etapa consiste em uma série de requisições para o endpoint `/movie/{movie_id}`, que retorna os dados detalhados de cada filme

###### Todos os endpoints utilizados nessa extração estão detalhados na [documentação oficial da API do TMDB](https://developer.themoviedb.org/reference/intro/getting-started)

In [65]:
import requests
import json
import pandas as pd
from os import environ as env
from concurrent.futures import ThreadPoolExecutor, as_completed


# API credentials config -----------------------------------------------------------------------------------------------------
API_AUTH_CONFIG = {
    "tmdb_api_token": env.get("TMDB_API_TOKEN") or "tmdb_api_token_default",
    "tmdb_api_key": env.get("TMDB_API_KEY") or "tmdb_api_key_default"
}

# API URLs config -----------------------------------------------------------------------------------------------------
index_url = "https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page=1&sort_by=popularity.desc"

# Pandas configs
pd.set_option('display.max_colwidth', None)

### Passo 1. Requisição dos dados de índice dos filmes

| Componente | Detalhe | Finalidade |
| :--- | :--- | :--- |
| **Endpoint** | **`/discover/movie`** | Obtém a lista de filmes que se enquadram em critérios amplos (ex: por popularidade, gênero, ano de lançamento). |
| **Filtros** | (Especifique os parâmetros usados, ex: `with_genres`, `sort_by`, `page`) | Define o universo de filmes a ser coletado. |
| **Output Crucial** | **`id`** (Identificador único do filme) | O ID é o dado chave que será utilizado na próxima etapa. |

In [66]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {API_AUTH_CONFIG['tmdb_api_token']}"
}

movie_index_response = requests.get(index_url, headers=headers)
movie_index_string = movie_index_response.content.decode('utf-8')
movies_json = json.loads(movie_index_string)

In [67]:
print(f'Total pages: {movies_json["total_pages"]}')
print(f'Total results: {movies_json["total_results"]}')
print(1053514/52676)

Total pages: 52676
Total results: 1053514
19.999886096134862


In [68]:
movie_index_df = pd.DataFrame(movies_json["results"])

print(movie_index_df)
print(len(movie_index_df))

    adult                     backdrop_path            genre_ids       id  \
0   False  /3IgJReIyunq95ta86CTSOs9vAht.jpg            [878, 12]   617126   
1   False  /1RgPyOhN4DRs225BGTlHJqCudII.jpg     [16, 28, 14, 53]  1311031   
2   False  /iZLqwEwUViJdSkGVjePGhxYzbDb.jpg            [878, 53]   755898   
3   False  /9KSboWOt09J72aMY4x8SS1IaOHK.jpg     [16, 28, 12, 14]   987400   
4   False  /aIsjCdfiAS89cMjdDWEpKmUTHsZ.jpg          [28, 10752]  1009640   
5   False  /fq8gLtrz1ByW3KQ2IM3RMZEIjsQ.jpg                 [27]  1038392   
6   False  /mEW9XMgYDO6U0MJcIRqRuSwjzN5.jpg             [28, 53]  1007734   
7   False   /Q2OajDi2kcO6yErb1IAyVDTKMs.jpg           [27, 9648]  1078605   
8   False  /bwP4UJZjjfAVOuTAWT8W8J2AKtG.jpg             [53, 28]  1450529   
9   False  /eU7IfdWq8KQy0oNd4kKXS0QUR08.jpg        [878, 12, 28]  1061474   
10  False  /xSD0q1FiuZkvHuy7uscOLbmd1hR.jpg         [12, 28, 14]    13494   
11  False  /or8y8JFF0vR3N9ap0Vdhf9tfTxQ.jpg             [28, 80]  1028248   

### Passo 2: Detalhamento Individual dos Filmes 

| Componente | Detalhe | Finalidade |
| :--- | :--- | :--- |
| **Endpoint** | **`/movie/{movie_id}`** | Realiza uma consulta específica para cada `id` obtido no Passo 1. |
| **Processo** | O código **itera** sobre a lista de IDs extraída. | Garante que cada filme tenha seus dados detalhados coletados. |
| **Output Final** | Dados detalhados (`budget`, `revenue`, `runtime`, `production_companies`, etc.). | Cria o *dataset* final para a análise. |

In [ ]:

# Set current search movie id list from index table response
movie_ids = movie_index_df['id'].tolist()

# Function for single movie request
def fetch_film(movie_id):
    detail_url = f'https://api.themoviedb.org/3/movie/{movie_id}'
    
    params = {
        "api_key": API_AUTH_CONFIG['tmdb_api_key'],
        # "append_to_response": "genres"
    }
    
    movie_detail_response = requests.get(detail_url, params=params)
    movie_detail_string = movie_detail_response.content.decode('utf-8')
    movie_detail_json = json.loads(movie_detail_string)
    
    print(f"Fetched: {movie_detail_json['title']}")
    
    return movie_detail_json

# Function for parallel movie detail requests
def fetch_movies_parallel(movie_ids, max_workers=5):
    movie_detail_results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_film, mid): mid for mid in movie_ids}
        for future in as_completed(futures):
            try:
                movie_detail_results.append(future.result())
            except Exception as e:
                mid = futures[future]
                print(f"Erro ao buscar filme {mid}: {e}")
    return movie_detail_results

movie_detail_data = fetch_movies_parallel(movie_ids)
print(f"\nTotal de filmes buscados: {len(movie_detail_data)}")

### Passo 3. Gerar as dataframes finais para a camada bronze

Alguns dos campos retornados são arrays de objetos. Esses irão se tornar dataframes separadas, interligadas pelo ID do filme. Em seguida cada uma dessas dataframes, se tornarão um csv para a camada bronze (raw data).

Os campos identificados como array são:
1. `genres`
2. `production_companies`
3. `production_countries`
4. `spoken_languages`

Nesse contexto, os CSVs de saída dessa extração serão:
1. movies.csv
2. genres.csv
3. production_companies.csv
4. production_countries.csv
5. spoken_languages.csv

A chave extrangeira que une cada um desses artefatos, é o ID do filme. Para que isso seja garantido, será adicionada uma coluna `movie_id` em cada nos demais CSVs (com exceção de movies.csv por ser o artefato "base").

In [77]:
array_fields = ["genres", "production_companies", "production_countries", "spoken_languages"]
regular_keys = [key for key in movie_detail_data[0] if key not in array_fields]

regular_movie_fields = []

for movie in movie_detail_data:
    current_movie = {k: v for k, v in movie.items() if k not in array_fields}
    regular_movie_fields.append(current_movie)
    
movie_detail_df = pd.DataFrame(regular_movie_fields)


In [84]:
array_fields_df = {}

for key in array_fields:
    current_df = pd.json_normalize(
        data=movie_detail_data,
        record_path=key,
        meta=['id'],
        record_prefix=f'{key}_'
    )
    current_df = current_df.rename(columns={'id': 'movie_id', f'{key}_id': 'id'})
    array_fields_df[f'{key}_df'] = current_df
    
    
genres_df = array_fields_df['genres_df']
production_companies_df = array_fields_df['production_companies_df']
production_countries_df = array_fields_df['production_countries_df']
spoken_languages_df = array_fields_df['spoken_languages_df']

### Passo 4. Salvar os CSVs na camada bronze

Nesse momento existem 5 dataframes, que serão salvas em arquivos CSVs na camada bronze da seguinte forma:

|Dataframe|Descrição|Arquivo de saída|
|---|---|---|
|movie_detail_df|Dataframe principal com os dados de detalhes de filmes|movies.csv|
|genres_df|Dataframe com dados de gênero dos filmes|genres.csv|
|production_companies_df|Dataframe com dados de produtoras de filmes|production_companies.csv|
|production_countries_df|Dataframe com dados de países da produção dos filmes|production_countries.csv|
|spoken_languages_df|Dataframe com os idiomas falados para cada filme|spoken_languages.csv|

In [86]:
movie_detail_df.to_csv('bronze/movies.csv', index=False)
genres_df.to_csv('bronze/genres.csv', index=False)
production_companies_df.to_csv('bronze/production_companies.csv', index=False)
production_countries_df.to_csv('bronze/production_countries.csv', index=False)
spoken_languages_df.to_csv('bronze/spoken_languages.csv', index=False)

## 3. Ferramentas e Bibliotecas utilizadas

| Ferramenta | Uso no Notebook |
| :--- | :--- |
| **Python `requests`** | Responsável por todas as chamadas HTTP para a API do TMDB. |
| **JSON** | Manipulação e *parsing* das respostas da API, que são formatadas em JSON. |
| **Pandas** | Estruturação e armazenamento dos dados extraídos em formato DataFrame. |